In [19]:
!pip install -qq selenium webdriver_manager

In [68]:
# 1. 모듈 임포트
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys
import requests

# 폴더 관리를 위한 모듈(5번 항목)
import os

# 이미지 링크 주소 파싱을 위한 모듈(6번 항목)
from urllib.parse import urlparse, unquote

In [69]:
# 2. 드라이버 옵션 설정 및 브라우저 띄우기
chrom_options = webdriver.ChromeOptions()
chrom_options.add_experimental_option('detach', True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                          options=chrom_options)
time.sleep(1)

In [70]:
# 3-1. 구글 이미지 검색 주소 이동
url = "https://www.google.com/imghp?hl=ko&tab=wi"
driver.get(url)
time.sleep(2)

In [71]:
# 3-2. 검색어 입력 및 검색
img_name = "오예스"
search_img = driver.find_element(By.CSS_SELECTOR, "textarea.gLFyf")
search_img.send_keys(img_name)
time.sleep(1)
search_img.send_keys(Keys.ENTER)

In [73]:
# 4-1. 처음 이미지 선택(선택해서 클릭한 이미지가 원본 이미지)
first_img = driver.find_element(By.CSS_SELECTOR, "#dimg_17")
first_img.click()
time.sleep(1)

In [74]:
# 4-2. 원본 이미지 링크 확인
target_img = driver.find_element(By.CSS_SELECTOR, "img.sFlh5c.pT0Scc.iPVvYb")
target_img_link = target_img.get_attribute("src")
print(target_img_link)

https://i.namu.wiki/i/MsPw7_U9Cm1sbwnrvpQHXyyMERvEljHG8M2q-LpksVuWgSBxFfqtp4j_HNB1nNtJ1GMzCPsQ2qKJuP6YXxSwiQ.webp


In [75]:
# 5. 목표 사진 개수 설정 및 저장 폴더 설정
target_count = 300
save_dir = f"./{img_name}/"

if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)

print(f"'{save_dir}' 폴더가 생성되었거나 이미 존재합니다.")

'./오예스/' 폴더가 생성되었거나 이미 존재합니다.


In [76]:
# 6-1. 다음 이미지 버튼 요소 추출 테스트
next_div = driver.find_elements(By.CSS_SELECTOR, "div.HJRshd")
# 처음 이미지 일때는 2개이며, 원하는 요소는 0번째 
# 다음 이미지일 때는 버튼이 3개, 원하는 요소는 1번째 따라서 인덱스는 뒤에서 -2
target_div = next_div[-2]

In [77]:
# 6-2. 버튼 테스트
buttons = target_div.find_elements(By.TAG_NAME, "button")
# 버튼 확인용
for btn in buttons:
    print(btn.get_attribute("aria-label"))

이전 이미지
다음 이미지
이 검색 결과에 관한 작업 더보기
닫기


In [78]:
# 6-3. 이미지 저장 링크 테스트
target_img = driver.find_elements(By.CSS_SELECTOR, "#Sva75c > div.A8mJGd.NDuZHe.OGftbe-N7Eqid-H9tDt > div.LrPjRb > div.AQyBn > div.tvh9oe.BIB1wf > c-wiz > div > div > div > div > div.v6bUne > div.p7sI2.PUxBg > a > img.sFlh5c.pT0Scc.iPVvYb")
for img in target_img:
    print("이미지 링크입니다", img.get_attribute("src"))

이미지 링크입니다 https://i.namu.wiki/i/MsPw7_U9Cm1sbwnrvpQHXyyMERvEljHG8M2q-LpksVuWgSBxFfqtp4j_HNB1nNtJ1GMzCPsQ2qKJuP6YXxSwiQ.webp


In [79]:
# 6-4 이미지 저장 테스트 - 이미지 데이터를 GET 요청으로 가져오기
target_img_link = target_img[-1].get_attribute("src")
response = requests.get(target_img_link)

# 요청이 성공적인 경우, 이미지를 로컬 파일로 저장
if response.status_code == 200:
    # 이미지 확장자 확인 및 쿼리 파라미터 제거를 위한 작업
    path = urlparse(target_img_link).path
    decoded_path = unquote(path)
    file_name = decoded_path.split("/")[-1]
    file_ext = file_name.split(".")[-1]
    save_name = f"{save_dir}{img_name}_img.{file_ext}"
    # 바이너리 쓰기 모드로 파일 열기
    with open(save_name, 'wb') as file:
        file.write(response.content)
    print(f"이미지가 성공적으로 다운로드되었습니다: {save_name}")
else:
    print("이미지 다운로드 실패")

이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_img.webp


In [80]:
# 6-5. 최종 : 다음 이미지 클릭하며, 원본 이미지 저장
for count in range(target_count):
    try:
        # 다음 이미지 버튼 요소 가져오기
        next_div = driver.find_elements(By.CSS_SELECTOR, "div.HJRshd")
        target_div = next_div[-2] 

        # 다음 이미지 버튼 클릭
        buttons = target_div.find_elements(By.TAG_NAME, "button")
        next_btn = buttons[1]
        next_btn.click() # 다음 이미지 버튼 클릭

        time.sleep(2)

        # target 이미지 선택
        target_img = driver.find_elements(By.CSS_SELECTOR, "img.sFlh5c.pT0Scc.iPVvYb")[-1]
        target_img_link = target_img.get_attribute("src")
        print(target_img_link)

        # 이미지 데이터를 GET 요청으로 가져오기 - header 정보를 넣어 주는 것이 좋습니다.
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(target_img_link, headers=headers)
        # 요청이 성공적인 경우, 이미지를 로컬 파일로 저장
        if response.status_code == 200:
            # 이미지 확장자 확인 및 쿼리 파라미터 제거를 위한 작업
            path = urlparse(target_img_link).path
            decoded_path = unquote(path)
            file_name = decoded_path.split("/")[-1]
            file_ext = file_name.split(".")[-1]

            if file_ext[0] == 'j' | file_ext[0] == 'p' | file_ext[0] == 'w':
                 file_ext = file_ext
            else:
                 file_ext = "jpg"

            save_name = f"{save_dir}{img_name}_{count}.{file_ext}"
            # 바이너리 쓰기 모드로 파일 열기
            with open(save_name, 'wb') as file:
                file.write(response.content)
            print(f"이미지가 성공적으로 다운로드되었습니다: {save_name}")
        else:
            print(f"{response.status_code}_이미지 다운로드 실패")
        time.sleep(1)

    except requests.exceptions.ConnectionError as e:
            print(f"Connection Pool 예외 발생: {e}")
            # 예외 발생 시 잠시 대기하고 다시 시도
            time.sleep(5)
    except Exception as e:
            print(f"예상치 못한 오류 발생: {e}")
    finally:
        time.sleep(1)

https://img.danawa.com/prod_img/500000/098/117/img/10117098_1.jpg?_v=20191210163932
이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_0.jpg
https://i.namu.wiki/i/Cq3LVUqLTtiMvYMK7oujVJMdDAgtWFNGT-Clk7rF3-OjiGz2PDcFko4YiC4g_3p_9t0UJ1zWJWqU8MULECDStw.webp
이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_1.webp
https://contents.lotteon.com/itemimage/LO/10/00/03/56/21/_1/00/00/35/62/2/LO1000035621_1000035622_1.jpg/dims/resizef/720X720
이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_2.720X720
https://upload.wikimedia.org/wikipedia/commons/b/b4/%EC%98%A4%EC%98%88%EC%8A%A4_%EC%82%AC%EC%A7%84.jpg
이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_3.jpg
https://cdn.011st.com/11dims/resize/600x600/quality/75/11src/product/2168415783/B.jpg?340000000
이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_4.jpg
https://images.emarteveryday.co.kr/images/app/webapps/evd_web2/share/SKU/mall/77/81/8801019318177_1.jpg
이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_5.jpg
https://sitem.ssgcdn.com/60/78/06/item/0000009067860_i1_750.jpg
이미지가 성공적으로 다운로드되었습니다: ./오예스/오예스_6.jpg
https://static.megamart.com/product/ima

KeyboardInterrupt: 